[1] 모듈 로딩 <hr>

In [8]:
## [1-1] 모듈 로딩
import numpy as np
import pandas as pd

## -> ML 관련 모듈
## 학습 알고리즘 모듈
from sklearn.neighbors import KNeighborsClassifier               

## 성능 평가 관련
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## 데이터 학습 준비 관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
irisDF = pd.read_csv('../DATA/iris.csv')

[2] 데이터 학습 준비 <hr>

In [10]:
## [2-1] 피쳐와 타겟, 학습용과 테스트용 데이터셋 분리
featureDF = irisDF[irisDF.columns[:-1]] 
targetSR = irisDF[irisDF.columns[-1]]
print(f'featureDF : {featureDF.shape}, targetSR : {targetSR}')

featureDF : (150, 4), targetSR : 0         Setosa
1         Setosa
2         Setosa
3         Setosa
4         Setosa
         ...    
145    Virginica
146    Virginica
147    Virginica
148    Virginica
149    Virginica
Name: variety, Length: 150, dtype: object


In [11]:
## -> 학습용/테스트용 데이터셋 분리
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR,
                                                    test_size = 0.2,
                                                    random_state=42,
                                                    stratify=targetSR)
print(f'[TRAIN] : {x_train.shape}, {y_train.shape}')
print(f'[TEST] : {x_test.shape}, {y_test.shape}')

[TRAIN] : (120, 4), (120,)
[TEST] : (30, 4), (30,)


In [12]:
## [2-2] 타겟 컬럼 수치화 : LabelEncoder
lbEncoder = LabelEncoder()

encode_y_train = lbEncoder.fit_transform(y_train)
encode_y_test = lbEncoder.transform(y_test)

print(f'classes_ : {lbEncoder.classes_}')


classes_ : ['Setosa' 'Versicolor' 'Virginica']


In [13]:
## [2-3] 수치형 피쳐 컬럼 : 스케일링
from sklearn.preprocessing import RobustScaler, StandardScaler

# => 평균 0, 분산 1이 되도록 스케일링 : 이상치 민감
stdScaler = StandardScaler()
std_x_train = stdScaler.fit_transform(x_train)
std_x_test = stdScaler.transform(x_test)

# => 중간/중앙값으로 스케일링 : 이상치 강함
rbScaler = RobustScaler()
rb_x_train = rbScaler.fit_transform(x_train)
rb_x_test = rbScaler.transform(x_test)



[3] 학습 <hr>

In [18]:
## GridSearchCV 기반 교차검증 진행 => 일반화 성능 체크
#   => 교차검증했을 때 성능이 가장 좋은 조합을 찾아서 알려주는 도구
from sklearn.model_selection import GridSearchCV


## 필요준비 : 모델 인스턴스, 모델의 하이퍼파라이머 dict
knn     = KNeighborsClassifier()
params  = {'n_neighbors':[3,5,7]}

## 교차검증 및 최적의 하이퍼파라미터 찾기 진행
search = GridSearchCV(knn,                      ## 무엇을 튜닝할지 정하는 모델 객체
                      params,                   ## 원하는 파라미터 목록
                      return_train_score=True,  ## 교차검증할 때 train 점수도 같이 저장할지 설정.
                      scoring='f1_micro')       ## 무슨 기준으로 성능을 평가할지 정하는 옵션.

## => 학습용 데이터셋으로 교차검증 진행 => 데이터 누수 (CV)
search.fit(std_x_train, encode_y_train)

## => 결과
print('cv_result_', search.cv_results_.keys())
print('best_score_', search.best_score_)
print('best_params_', search.best_params_)

cv_result_ dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])
best_score_ 0.9666666666666668
best_params_ {'n_neighbors': 5}


In [19]:
## [3-2] 최적의 하이퍼파라미터 적용 + 전체 학습데이터로 학습 진행
BEST_N = search.best_params_['n_neighbors']

final_model = KNeighborsClassifier(n_neighbors=BEST_N)
final_model.fit(std_x_train, encode_y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[4]  다양한 분류용 성능지표기반 평가 <hr>

In [ ]:
## 성능지표함수(1D_정답, 1D_예측)
pre_y = final_model.predict(std_x_test)

## 성능지표 : 정확도
acc_1 = accuracy_score(encode_y_test, pre_y) # accuracy_score(정답, 예측)
acc_2 = final_model.score(std_x_test, encode_y_test)
print(f'accuracy_score() : {acc_1}, score() 메서드 : {acc_2}')

accuracy_score() : 0.9333333333333333, score() 메서드 : 0.9333333333333333


In [24]:
## 성능지표 : 정밀도
precision_ = precision_score(encode_y_test, pre_y) # accuracy_score(정답, 예측)

print(f'accuracy_score() : {acc_1}, score() 메서드 : {acc_2}')

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].